In [1]:
from konlpy.tag import Komoran
komoran=Komoran()
import json
import pandas as pd

In [3]:
import tensorflow as tf

In [2]:
def preprocess_text(complaint, word_index, max_sequence_length):
    # Tokenize the complaint into morphemes (words)
    tokenized_complaint = komoran.morphs(complaint)  # Implement your tokenization function

    # Convert morphemes to word indices using the word_index dictionary
    indexed_complaint = [word_index[word] for word in tokenized_complaint if word in word_index]

    # Pad or truncate the sequence to match max_sequence_length
    padded_complaint = tf.keras.preprocessing.sequence.pad_sequences([indexed_complaint],
                                                                     maxlen=max_sequence_length,
                                                                     padding='post')

    return padded_complaint

In [14]:
def map_index_to_category(predicted_index):
    categories = {0: '인사', 1: '작별인사', 2: '대건고 위치',
                  3: '대건고 교훈', 4: '대건고 교목', 5: '대건고 교화'}

    return categories[predicted_index]


In [25]:
def predict_category(model, complaint, word_index, max_sequence_length):
    # Preprocess the complaint
    padded_complaint = preprocess_text(complaint, word_index, max_sequence_length)

    # Make predictions using the model
    predictions = model.predict(padded_complaint)

    # Get the index of the category with the highest probability
    predicted_category_index = tf.argmax(predictions[0]).numpy()

    print(predicted_category_index)
    
    category = map_index_to_category(predicted_category_index)

    return category


In [6]:
def chatbot_response(user_input, model):
    predicted_category_index = predict_category(user_input, model)
    predicted_category = map_index_to_category(predicted_category_index)

    response = f"Thank you for your complaint. It falls under the category: {predicted_category}. We will look into it."

    return response


In [7]:
import pickle

In [26]:
# Assuming you have already loaded the trained model and word_index dictionary
model = tf.keras.models.load_model('Chatbot_Model.h5')
with open('Word_Dict.pkl','rb') as f:
   word_index = pickle.load(f)
#word_index = {'한국어': 1, '자연어': 2, ...}  # Your word_index dictionary

# User input complaint
user_complaint = "안녕"
max_sequence_length = 10

# Preprocess the complaint and get the padded sequence
category = predict_category(model, user_complaint, word_index, max_sequence_length)

# Make predictions using the model
#predictions = model.predict(padded_complaint)


1/1 [==============================] - 0s 255ms/step
5


In [24]:
category

'대건고 교화'